In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier,Pool
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer

In [2]:
import pandas as pd

# 최대 줄 수 설정
pd.set_option('display.max_rows', 500)

# 최대 열 수 설정
pd.set_option('display.max_columns', 500)

# 표시할 가로의 길이
pd.set_option('display.width', 1000)

In [3]:
# 데이터 불러오기
train_dataset = pd.read_csv("../Dataset/train.csv")
test_dataset = pd.read_csv("../Dataset/test.csv")
train_dataset.head()

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,country,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,1.0,5.0,1.0,5.0,1.0,1.0,USA,3,553,6,4.0,3.0,5.0,1.0,3.0,5.0,5.0,3.0,5.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,2.0,1,3.0,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,1.0,4.0,5.0,3.0,1.0,2.0,4.0,5.0,1.0,3.0,1.0,1.0,5.0,3.0,2.0,5.0,USA,5,85,120,4.0,2.0,3.0,5.0,3.0,2.0,5.0,1.0,2.0,2.0,1,1,1,1,1,0,1,0,0,1,0,0,1,1,1,1,4.0,2,2.0,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,2.0,5.0,5.0,5.0,1.0,3.0,5.0,3.0,5.0,2.0,2.0,1.0,2.0,4.0,2.0,5.0,NLD,9,108,100,1.0,2.0,3.0,1.0,5.0,5.0,3.0,4.0,5.0,2.0,1,1,0,1,1,0,1,1,0,1,0,0,1,1,1,1,2.0,1,1.0,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,5.0,2.0,2.0,4.0,4.0,2.0,4.0,5.0,4.0,3.0,3.0,4.0,3.0,4.0,4.0,2.0,USA,2,121,139,3.0,3.0,3.0,4.0,5.0,3.0,4.0,4.0,3.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,1.0,3,1.0,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,4.0,4.0,3.0,5.0,5.0,2.0,4.0,1.0,4.0,2.0,4.0,2.0,3.0,4.0,4.0,4.0,ITA,3,640,216,3.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,2.0,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,1,1.0,2,2.0,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0


### - Preprocessing
1. null 값
    - Q1~ Q26 , TIPI1 ~ TIPI10  : 0으로 대체
    - 
2. 이상치(나이, familysize)
    - age : 123, 336, 545, 722, 38822  ( age > 100 인 값 제거)
    - familysize : 2919 제거
3. country 아예 제거

In [4]:
# 전처리 데이터는 따로 관리!
processed_train = train_dataset.drop(['index'], axis=1).copy()
processed_test = test_dataset.drop(['index'], axis =1).copy()

In [5]:
question_col = ['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8','Q9','Q10','Q11','Q12',
               'Q13','Q14','Q15','Q16','Q17','Q18', 'Q19','Q20','Q21',
                'Q22','Q23','Q24','Q25','Q26']

tipi_col = ['TIPI1','TIPI2','TIPI3','TIPI4',
            'TIPI5','TIPI6','TIPI7','TIPI8','TIPI9','TIPI10']

vcl_col = ['VCL1', 'VCL2', 'VCL3', 'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL', 'VCL9'
           'VCL10', 'VCL11', 'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16']

replace_type = {'education': str, 'engnat': str, 'married': str, 'urban': str, 'gender' : str, 'religion' : str,
               'orientation' : str, 'voted': str, 'ASD':str}

In [6]:
# processed_train = processed_train.drop(remove_col, axis=1)
# processed_test = processed_test.drop(remove_col, axis=1)

In [7]:
# 100 살이 넘는 애들은 없애
remove_row1 = processed_train[processed_train['age'] > 100].index
processed_train = processed_train.drop(remove_row1)

In [8]:
# familysize가 말도 안되게 큰 2919도 없애
remove_row2 = processed_train[processed_train['familysize'] == 2919].index
processed_train= processed_train.drop(remove_row2)

In [9]:
# ASD 결측치 값은 다 삭제한담에 해볼게요
processed_train = processed_train.dropna(subset = ['ASD'])

In [10]:
# 나머지 결측치는 simple 중에 가장 많이 나오는 애들로 대체하는 방법으로 해보겠습니다
simple_imp = SimpleImputer(missing_values = np.nan, strategy='most_frequent')

processed_train = pd.DataFrame(simple_imp.fit_transform(processed_train), columns = processed_train.columns, index = processed_train.index)
processed_test = pd.DataFrame(simple_imp.fit_transform(processed_test), columns = processed_test.columns, index = processed_test.index)

In [11]:
# dummy 변수를 만드는 방법으로 범주형 전처리를 해보도록 할게요
# education, engnat, married, urban, orientation, voted, ASD
processed_train = processed_train.astype(replace_type)
processed_test = processed_test.astype(replace_type) 

In [12]:
processed_train = pd.get_dummies(processed_train)
processed_test = pd.get_dummies(processed_test)

In [15]:
processed_train.info

<bound method DataFrame.info of        Q1_1.0  Q1_2.0  Q1_3.0  Q1_4.0  Q1_5.0  Q2_1.0  Q2_2.0  Q2_3.0  Q2_4.0  Q2_5.0  Q3_1.0  Q3_2.0  Q3_3.0  Q3_4.0  Q3_5.0  Q4_1.0  Q4_2.0  Q4_3.0  Q4_4.0  Q4_5.0  Q5_1.0  Q5_2.0  Q5_3.0  Q5_4.0  Q5_5.0  Q6_1.0  Q6_2.0  Q6_3.0  Q6_4.0  Q6_5.0  Q7_1.0  Q7_2.0  Q7_3.0  Q7_4.0  Q7_5.0  Q8_1.0  Q8_2.0  Q8_3.0  Q8_4.0  Q8_5.0  Q9_1.0  Q9_2.0  Q9_3.0  Q9_4.0  Q9_5.0  Q10_1.0  Q10_2.0  Q10_3.0  Q10_4.0  Q10_5.0  Q11_1.0  Q11_2.0  Q11_3.0  Q11_4.0  Q11_5.0  Q12_1.0  Q12_2.0  Q12_3.0  Q12_4.0  Q12_5.0  Q13_1.0  Q13_2.0  Q13_3.0  Q13_4.0  Q13_5.0  Q14_1.0  Q14_2.0  Q14_3.0  Q14_4.0  Q14_5.0  Q15_1.0  Q15_2.0  Q15_3.0  Q15_4.0  Q15_5.0  Q16_1.0  Q16_2.0  Q16_3.0  Q16_4.0  Q16_5.0  Q17_1.0  Q17_2.0  Q17_3.0  Q17_4.0  Q17_5.0  Q18_1.0  Q18_2.0  Q18_3.0  Q18_4.0  Q18_5.0  Q19_1.0  Q19_2.0  Q19_3.0  Q19_4.0  Q19_5.0  Q20_1.0  Q20_2.0  Q20_3.0  Q20_4.0  Q20_5.0  Q21_1.0  Q21_2.0  Q21_3.0  Q21_4.0  Q21_5.0  Q22_1.0  Q22_2.0  Q22_3.0  Q22_4.0  Q22_5.0  Q23_1.0  Q23_2.0

### - Data Split

In [ ]:
train_data = processed_train.copy()

input_y = train_data['nerdiness'].copy()
input_X = train_data.drop(['nerdiness'], axis=1)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(
    input_X, 
    input_y, 
    test_size = 0.2, 
    random_state = 3,
    shuffle=True
)

### - Modeling AdaBoost

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split

In [ ]:
kfold = KFold(n_splits=5, shuffle = True, random_state = 3)
cv_auc=[]

ada_model = AdaBoostClassifier(n_estimators = 100, 
                               random_state=10, learning_rate=0.1)

n_iter = 0 

for train_index, test_index in kfold.split(train_X):
  # kfold.split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
    Xtrain, Xval = train_X.iloc[train_index], train_X.iloc[test_index]
    ytrain, yval = train_y.iloc[train_index], train_y.iloc[test_index]
    # 학습 및 예측
    ada_model.fit(Xtrain, ytrain)
    pred_fold = ada_model.predict(Xval)
    n_iter += 1

    auc = roc_auc_score(yval, pred_fold)
    train_size = Xtrain.shape[0]
    test_size = Xval.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'.format(n_iter, auc, train_size, test_size))
    print('#{0} 검증 세트 인덱스 :{1}'.format(n_iter, test_index))
    cv_auc.append(auc)

# 개별 iteration별 정확도를 합하여 평균 정확도 계산
print('\n## 평균 검증 정확도:', np.mean(cv_auc))

In [ ]:
model_pred = ada_model.predict(test_X)
prob_pred = ada_model.predict_proba(test_X)[:,1] #proba를 통해 확률값으로 변환
print(roc_auc_score(test_y, model_pred))
print(roc_auc_score(test_y, prob_pred))

In [ ]:
ftr_importances_values = ada_model.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index=train_X.columns)
ftr_top15 = ftr_importances.sort_values(ascending=False)[:15]

plt.figure(figsize=(8,6))
plt.title('Feature importances Top 15')
sns.barplot(x = ftr_top15, y = ftr_top15.index)
plt.show()

### Submission Model

In [ ]:
test = processed_test.copy()
ada_model_pred = ada_model.predict_proba(test)[:, 1]

In [ ]:
submission = pd.read_csv("../Dataset/sample_submission.csv")
submission

In [ ]:
submission["nerdiness"] = ada_model_pred

In [ ]:
submission

In [ ]:
submission.to_csv("../Dataset/Adaboost.csv", index = False)